In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
# #PATH

path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_file = "F:/178/Tugas Akhir/dataset/clips_45sec_wav/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"

In [7]:
datafitur = "mfcc"
data = pd.read_csv(path_to_experiment + "mfcc.csv",header=None)
ground = pd.read_csv(path_info + "static_annotations.csv")
arousal_mean = ground['mean_arousal']
valence_mean = ground['mean_valence']
arousal_std = ground['std_arousal']
valence_std = ground['std_valence']

data = np.asarray(data)
data = data.T
# arousal_mean = np.asarray(arousal_mean)
# # arousal_mean = arousal_mean.reshape((744,1))
# valence_mean = np.asarray(valence_mean)
# # valence_mean = valence_mean.reshape((744,1))

In [9]:
print(data.shape)
print(arousal_mean.shape)

(744, 25194)
(744,)


In [71]:
svr = SVR(kernel='rbf',gamma =1e-8)

In [72]:
train_data, test_data, train_values, test_values = train_test_split(data,arousal_mean,test_size=0.3,random_state=100)

In [73]:
svr.fit(train_data,train_values)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-08,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [74]:
a=svr.predict(test_data)

In [75]:
a_round = a.round(1)

In [76]:
# np.shape(a)
df = pd.DataFrame(data=[a,a_round,test_values])
df

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,3.41975,5.6114,3.650733,5.043911,4.29997,3.914781,5.310629,2.288478,3.931355,4.537215,...,3.274854,5.369446,4.160836,5.090504,4.53255,4.640471,5.292814,3.785813,4.004905,5.491552
1,3.40000,5.6000,3.700000,5.000000,4.30000,3.900000,5.300000,2.300000,3.900000,4.500000,...,3.300000,5.400000,4.200000,5.100000,4.50000,4.600000,5.300000,3.800000,4.000000,5.500000
2,2.80000,7.3000,4.900000,4.800000,4.40000,2.900000,4.400000,3.200000,3.900000,4.800000,...,3.400000,4.400000,4.600000,3.900000,4.80000,5.100000,5.300000,4.200000,4.300000,7.100000


In [77]:
r2 = r2_score(test_values, a)
r2_round = r2_score(test_values, a_round)

print(r2)
print(r2_round)

0.557322390817
0.555337619422


In [83]:
svr = SVR(kernel='rbf',gamma =1e-8,C=1.0)

scores_am = cross_val_score(svr,data,arousal_mean,scoring='r2',cv=10)
scores_as = cross_val_score(svr,data,arousal_std,scoring='r2',cv=10)
print("R2 Score Arousal: %0.2f (+/- %0.2f)" % (scores_am.mean(), scores_as.std() * 2))

scores_vm = cross_val_score(svr,data,arousal_mean,scoring='r2',cv=10)
scores_vs = cross_val_score(svr,data,arousal_std,scoring='r2',cv=10)
print("R2 Score Valence: %0.2f (+/- %0.2f)" % (scores_vm.mean(), scores_vs.std() * 2))

R2 Score Arousal: 0.53 (+/- 0.14)
R2 Score Valence: 0.53 (+/- 0.14)


In [27]:
nu_svr = NuSVR(kernel='rbf',gamma =1e-8)

scores = cross_val_score(nu_svr,data,valence_mean,scoring='r2',cv=10)
print("R2 Score Valence Mean: ",scores.mean())

scores = cross_val_score(nu_svr,data,arousal_mean,scoring='r2',cv=10)
print("R2 Score Arousal Mean: ",scores.mean())

scores=cross_val_score(nu_svr,data,valence_std,scoring='r2',cv=10)
print("R2 Score Valence STD: ",scores.std())

scores = cross_val_score(nu_svr,data,arousal_std,scoring='r2',cv=10)
print("R2 Score Arousal STD: ",scores.std())

R2 Score Valence Mean:  0.0937798637354
R2 Score Arousal Mean:  0.543303332205
R2 Score Valence STD:  0.0548129391169
R2 Score Arousal STD:  0.0717292935935
